In [ ]:
#!/usr/bin/env python
# Shebang line to execute script directly from command line

# Import necessary packages/modules
import random  # for generating random numbers in weighted_choice function
import re  # for regex-based tokenization of text as states
import textwrap  # for readability in generated text output to terminal and file

# defaultdict to auto-initialize nested dictionaries for ngram counts
# Counter to efficiently tally token frequencies
from collections import defaultdict, Counter


def read_text_file(filepath):
    """
    Function to open and read input text file line by line, remove whitespace,
    newline characters, and leading/trailing spaces, and return continuous
    cleaned string of text
        Parameters:
            filepath (string): Path to input file
        Returns:
            text (string): Single continuous string containing properly formatted text
    """

    # Initialize empty string to store processed text
    text = " "

    # Open file with context manager
    with open(filepath, "r") as file:

        # Iterate through file by line
        for line in file:

            # Strip leading/trailing whitespace and newline characters
            text += " " + line.strip()  # Add space before lines for line distinction

    # Return text string
    return text


def tokenize(text):
    """
    Helper function, converts raw text into lowercase tokens list
    Tokenizes words, contractions, hyphens, numbers, punctuation
        Parameters:
            text (string): Raw text to tokenize
        Returns:
            tokens (list of strings): List of extracted tokens
    """

    # Regex pattern to capture: alphabetic words, words with hyphens or apostrophes,
    # numbers, punctuations as individual tokens
    pattern = r"[A-Za-z]+(?:[-'][A-Za-z]+)*|\d+|[\"'()\-\.,!?;:]"

    # Convert text to lowercase, extract tokens with regex
    tokens = re.findall(pattern, text.lower())

    # Return tokens as list of strings
    return tokens


def word_frequencies(text):
    """
    Count times each token exists in input text
        Parameters:
            text (string): Raw input text
        Returns:
            freqs (Counter): Mapping of tokens to frequencies
    """

    # Convert raw text into list of tokens
    tokens = tokenize(text)

    # Counter auto-tallies each token occurrence
    freqs = Counter(tokens)

    # Return frequency dictionary
    return freqs


def word_probabilities(text):
    """
    Compute probability of tokens in text based on frequency of occurrence
        Parameters:
            text (string): Raw input text
        Returns:
            probs (dict): Mapping of tokens to probability values
    """

    # Get raw frequency counts
    freqs = word_frequencies(text)

    # Compute total number tokens
    total = sum(freqs.values())

    # Convert counts to probabilities
    probs = {token: count / total for token, count in freqs.items()}

    # Return probability dictionary
    return probs


def ngram_frequencies(text, order):
    """
    Count how often each n-gram state transitions to another token.
    Establishes structure of Markov chain by mapping
    states (tuple of length 'order') to dictionary of next-token counts
        Parameters:
            text (string): Raw input text
            order (int): Markov chain order (N)
        Returns:
            freqs (dict): Mapping of state tuples to dictionaries of next-token counts
    """

    # Tokenize raw text into tokens list
    tokens = tokenize(text)

    # Add start-state (beta) "*S*" repeated 'order' times with one end marker "*E*"
    tokens = ["*S*"] * order + tokens + ["*E*"]

    # Initialize nested dictionary:
    # outer dict: state tuple -> inner dict
    # inner dict: next_token -> count
    freqs = defaultdict(lambda: defaultdict(int))

    # Iterate through tokens to build ngram transitions
    for i in range(len(tokens) - order):

        # Extract state as tuple of previous 'order' tokens
        state = tuple(tokens[i : i + order])

        # Identify next token from current state
        next_token = tokens[i + order]

        # Increment count for transition
        freqs[state][next_token] += 1

    # Convert nested defaultdicts to normal dicts for clarity
    return {state: dict(next_tokens) for state, next_tokens in freqs.items()}


def build_markov_model(text, order):
    """
    Build Markov model from input text by computing ngram frequencies to generate
    text based on tokens from input text
        Parameters:
            text (string): Raw input text
            order (int): Markov chain order (N)
        Returns:
            model (dict): Mapping of state tuples to next-token counts
    """

    # Compute ngram transition frequencies
    model = ngram_frequencies(text, order)

    # Return model dictionary
    return model


def distribution_frequencies(items):
    """
    Count frequencies of any text states
        Parameters:
            items (list): List of hashable items
        Returns:
            freqs (dict): Mapping of items to associated frequencies
    """

    # Initialize dictionary to track occurrences
    freqs = defaultdict(int)

    # Count each item
    for item in items:
        freqs[item] += 1

    # Return normal dictionary
    return dict(freqs)


def distribution_probabilities(freqs):
    """
    Convert frequency dictionary into probability distribution
        Parameters:
            freqs (dict): Mapping of items to frequency counts
        Returns:
            probs (dict): Mapping of items to probabilities
    """

    # Compute total count
    total = sum(freqs.values())

    # Initialize probability dictionary
    probs = {}

    # Convert counts into probabilities
    for item, count in freqs.items():
        if total == 0:
            probs[item] = 0.0
        else:
            probs[item] = count / total

    # Return probability dictionary
    return probs


def beta_frequencies(text, order):
    """
    Count number of times starting state appears in text.
    For Nth-order Markov chains start state is first 'order' tokens
        Parameters:
            text (string): Raw input text
            order (int): Markov chain order (N)
        Returns:
            freqs (dict): Mapping of start-state tuples to frequencies
    """

    # Tokenize raw text
    tokens = tokenize(text)

    # Add start markers to simulate missing prior context
    tokens = ["*S*"] * order + tokens

    # Extract start state as tuple of first 'order' tokens
    start_state = tuple(tokens[:order])

    # Count start state
    freqs = distribution_frequencies([start_state])

    # Return frequency dictionary
    return freqs


def beta_probabilities(beta_freqs):
    """
    Convert start-state frequencies (beta) into start-state probabilities
        Parameters:
            beta_freqs (dict): Mapping of start-state tuples to frequencies
        Returns:
            beta_probs (dict): Mapping of start-state tuples to probabilities
    """

    # Transform frequencies into probabilities
    beta_probs = distribution_probabilities(beta_freqs)

    # Return probability dictionary
    return beta_probs


def omega_frequencies(markov_model):
    """
    Count how many times states transition to end marker "*E*"
        Parameters:
            markov_model (dict): Mapping of state tuples to next-token tallies
        Returns:
            omega_freqs (dict): Mapping of state tuples to ending frequencies
    """

    # Initialize dictionary for ending frequencies
    omega_freqs = {}

    # Iterate through all states in model
    for state, transitions in markov_model.items():

        # If state transitions to "*E*", record count
        if "*E*" in transitions:
            omega_freqs[state] = transitions["*E*"]
        else:
            # Otherwise, ending frequency is zero
            omega_freqs[state] = 0

    # Return dictionary of ending frequencies
    return omega_freqs


def omega_probabilities(omega_freqs):
    """
    Convert end-state frequencies (omega) into end-state probabilities
        Parameters:
            omega_freqs (dict): Mapping of state tuples to end-state frequencies
        Returns:
            omega_probs (dict): Mapping of state tuples to end probabilities
    """

    # Convert ending frequencies into probabilities
    omega_probs = distribution_probabilities(omega_freqs)

    # Return probability dictionary
    return omega_probs


def weighted_choice(options_dict):
    """
    Select token from dictionary of {token: probability} by applying
    cumulative probability weighting
        Parameters:
            options_dict (dict): Mapping of tokens to probability values
        Returns:
            token (string): Selected token based on weighted random choice
    """

    # Generate random float between 0 and 1
    random_float = random.random()

    # Track cumulative probability
    cumulative = 0.0

    # Iterate through tokens and their probabilities
    for token, prob in options_dict.items():

        # Add probability to cumulative total
        cumulative += prob

        # If random number falls within cumulative range, return token
        if random_float <= cumulative:
            return token

    # Fallback return
    return token


def get_next_word(state, model, omega_probs):
    """
    Given current state, predict next token based on transition probabs from Markov model,
    ending probabilities from omega
        Parameters:
            state (tuple): Current Markov state of length: order
            model (dict): Mapping of states to next-token counts
            omega_probs (dict): End-state probability dist
        Returns:
            next_token (string): Predicted next token
    """

    # Retrieve ending probability for state (default 0.0)
    end_prob = omega_probs.get(state, 0.0)

    # Initialize dictionary for next-token probabilities
    next_token_probs = {}

    # If state exists, compute transition probs
    if state in model:

        # Retrieve next-token counts
        transitions = model[state]

        # Compute total transitions from current state
        total = sum(transitions.values())

        # Convert counts to probs
        for token, count in transitions.items():
            next_token_probs[token] = count / total

    # Add ending prob as possible next token
    if end_prob > 0:
        next_token_probs["*E*"] = end_prob

    # Select next token using weighted random choice
    return weighted_choice(next_token_probs)


def compute_metadata(text):
    """
    Optional helper to compute metadata, incl average sentence length
        Parameters:
            text (string): Raw input text
        Returns:
            metadata (dict): Contains average sentence length
    """

    # Split text into sentences using punctuation
    sentences = re.split(r"[.!?]+", text)

    # Remove empty strings and strip whitespace
    sentences = [s.strip() for s in sentences if s.strip()]

    # If no sentences found, return default metadata
    if not sentences:
        return {"avg_length": 45}

    # Compute number of words per sentence
    word_counts = [len(s.split()) for s in sentences]

    # Compute average sentence length
    avg_length = round(sum(word_counts) / len(word_counts))

    # Return metadata dictionary
    return {"avg_length": avg_length}


def generate_random_text(model, beta_probs, omega_probs, order, max_words=150, metadata=None, use_metadata=False):
    """
    Generate random text sequence from Markov model application to input text
        Parameters:
            model (dict): Mapping of states to next-token counts
            beta_probs (dict): Start-state probability distribution
            omega_probs (dict): Ending-state probability distribution
            order (int): Markov chain order (N)
            max_words (int): Maximum tokens to generate
            metadata (dict): Optional metadata (avg sentence length)
            use_metadata (bool): Whether to use metadata to guide length

        Returns:
            generated_text (string): Generated text sequence
    """

    # Choose starting state using beta probabilities
    current_state = weighted_choice(beta_probs)

    # Initialize list to store generated tokens
    generated = []

    # Determine target length based on metadata or max_words
    if use_metadata and metadata:
        target_length = metadata["avg_length"]
    else:
        target_length = max_words

    # Generate up to target_length tokens
    for _ in range(target_length):

        # Predict next token
        next_token = get_next_word(current_state, model, omega_probs)

        # Stop if end marker reached
        if next_token == "*E*":
            break

        # Append token to output list
        generated.append(next_token)

        # Update state by shifting left and adding new token
        current_state = tuple(list(current_state[1:]) + [next_token])

    # Join tokens into single string
    return " ".join(generated)


def output_generated_text(text, filename, width=80):
    """
    Print generated text to terminal and save to output file using text wrapping
        Parameters:
            text (string): Text generated from Markov model based on input text style
            filename (string): Name of output file
            width (int): Maximum characters per line for wrapping
        Returns:
            None
    """

    # Wrap text for readability
    wrapped_text = textwrap.fill(text, width=width)

    # Print wrapped text to terminal
    print(wrapped_text)

    # Save wrapped text to output file
    with open(filename, "w") as f:
        f.write(wrapped_text)


if __name__ == "__main__":
    """
    Driver code to run Markov chain text generator
    """

    # Input text file from project directory
    input_file = "one_fish_two_fish.txt"

    # Markov chain order (N)
    order = 8

    # Read, prepare input text
    raw_text = read_text_file(input_file)

    # Build Markov model from input text
    model = build_markov_model(raw_text, order)

    # Compute beta (start-state) distribution
    beta_freqs = beta_frequencies(raw_text, order)
    beta_probs = beta_probabilities(beta_freqs)

    # Compute omega (end-state) distribution
    omega_freqs = omega_frequencies(model)
    omega_probs = omega_probabilities(omega_freqs)

    # Compute metadata (avg sentence length) if desired
    metadata = compute_metadata(raw_text)

    # Generate text from trained Markov model
    generated_text = generate_random_text(
        model=model,
        beta_probs=beta_probs,
        omega_probs=omega_probs,
        order=order,
        max_words=200,
        metadata=metadata,
        use_metadata=False
    )

    # Print and save generated text
    output_generated_text(generated_text, "generated_output.txt")
